In [1]:
import pandas as pd

In [37]:
large = True

In [38]:
results = pd.read_csv('./results_large.csv' if large else './results.csv', names=['name', 'repo', 'acc', '#w', 'samples', 'time', 'model', 'model_address', 'split'])
results.tail(1)

,name,repo,acc,#w,samples,time,model,model_address,split
161,normal,/data/all/data/foam-framework/foam,0.36,13,33,10-05-2022_22-27-39,t5-large,/scratch/arminz/tmp/bt_t5-large//data/all/data...,repo-based-included


In [39]:
results = results[results['split'] != 0.5]
results = results[results['model'] == f't5-{"large" if large else "small"}']

In [40]:
def extract_type_from_address(address):
    for keyword in ['bt', '_length_input', '_length_label', '_conf', 'included', 'ft', 'basic', 'finetuned', 'adapted']:
        if keyword in address:
            if keyword == 'ft' and '0.3_4' in address:
                return 'ft_0.3_4'
            return keyword
    return 'excluded'

In [41]:
results['type'] = results['model_address'].apply(extract_type_from_address)

In [42]:
results['type'].value_counts()

ft               30
finetuned        20
adapted          20
_conf            20
_length_label    20
_length_input    20
excluded         11
included         11
bt               10
Name: type, dtype: int64

In [43]:
# results[results['type'] == 'ft']

In [44]:
results.tail(3)

,name,repo,acc,#w,samples,time,model,model_address,split,type
159,normal,/data/all/data/LivelyKernel/LivelyKernel,0.64,12,36,10-05-2022_21-57-49,t5-large,/scratch/arminz/tmp/bt_t5-large//data/all/data...,repo-based-included,bt
160,normal,/data/all/data/svgdotjs/svg.js,0.10,5,31,10-05-2022_22-13-51,t5-large,/scratch/arminz/tmp/bt_t5-large//data/all/data...,repo-based-included,bt
161,normal,/data/all/data/foam-framework/foam,0.36,13,33,10-05-2022_22-27-39,t5-large,/scratch/arminz/tmp/bt_t5-large//data/all/data...,repo-based-included,bt


In [45]:
f'{10:.2f}, {1:.2f}'

'10.00, 1.00'

In [46]:
def print_stats(df):
    print(f"{len(df)}, {100 * (df['acc'] * df['samples']).sum() / df['samples'].sum():.2f}, {100 * df['acc'].mean():.2f}, {100 * df['acc'].median():.2f}")

In [47]:
print_stats(results[(results['type'] == 'included') & (results['repo'] != 'all')])

10, 63.76, 65.10, 61.50


In [48]:
print_stats(results[(results['type'] == 'excluded') & (results['repo'] != 'all')])

10, 49.20, 49.20, 45.50


In [49]:
print_stats(results[(results['type'] == 'finetuned') & (results['repo'] != 'all')])

10, 64.84, 66.30, 63.50


In [50]:
print_stats(results[(results['type'] == 'finetuned') & (results['repo'] == 'all')])

10, 46.50, 46.50, 46.00


In [51]:
print_stats(results[(results['type'] == 'adapted') & (results['repo'] != 'all')])

10, 65.87, 67.00, 61.50


In [52]:
print_stats(results[(results['type'] == 'adapted') & (results['repo'] == 'all')])

10, 43.90, 43.90, 44.00


In [53]:
print_stats(results[(results['type'] == 'ft') & (results['repo'] != 'all')])

20, 44.86, 42.85, 45.00


In [54]:
print_stats(results[(results['type'] == 'curr') & (results['repo'] != 'all')])

0, nan, nan, nan


/tmp/ipykernel_49347/2078206976.py:2: RuntimeWarning: invalid value encountered in double_scalars
  print(f"{len(df)}, {100 * (df['acc'] * df['samples']).sum() / df['samples'].sum():.2f}, {100 * df['acc'].mean():.2f}, {100 * df['acc'].median():.2f}")


In [55]:
print_stats(results[(results['type'] == 'curr') & (results['repo'] == 'all')])

0, nan, nan, nan


/tmp/ipykernel_49347/2078206976.py:2: RuntimeWarning: invalid value encountered in double_scalars
  print(f"{len(df)}, {100 * (df['acc'] * df['samples']).sum() / df['samples'].sum():.2f}, {100 * df['acc'].mean():.2f}, {100 * df['acc'].median():.2f}")


In [56]:
print_stats(results[(results['type'] == 'adapted') & (results['repo'] == 'all')])

10, 43.90, 43.90, 44.00


In [57]:
print_stats(results[(results['type'] == 'adapted') & (results['repo'] != 'all') & results['split'].isna()])

0, nan, nan, nan


/tmp/ipykernel_49347/2078206976.py:2: RuntimeWarning: invalid value encountered in double_scalars
  print(f"{len(df)}, {100 * (df['acc'] * df['samples']).sum() / df['samples'].sum():.2f}, {100 * df['acc'].mean():.2f}, {100 * df['acc'].median():.2f}")


In [58]:
print_stats(results[(results['type'] == 'adapted') & (results['repo'] != 'all') & ~results['split'].isna()])

10, 65.87, 67.00, 61.50


In [59]:
print_stats(results[(results['type'] == 'ft_0.3_4') & (results['repo'] != 'all')])

0, nan, nan, nan


/tmp/ipykernel_49347/2078206976.py:2: RuntimeWarning: invalid value encountered in double_scalars
  print(f"{len(df)}, {100 * (df['acc'] * df['samples']).sum() / df['samples'].sum():.2f}, {100 * df['acc'].mean():.2f}, {100 * df['acc'].median():.2f}")


In [60]:
print_stats(results[(results['type'] == 'ft_0.3_4') & (results['repo'] == 'all')])

0, nan, nan, nan


/tmp/ipykernel_49347/2078206976.py:2: RuntimeWarning: invalid value encountered in double_scalars
  print(f"{len(df)}, {100 * (df['acc'] * df['samples']).sum() / df['samples'].sum():.2f}, {100 * df['acc'].mean():.2f}, {100 * df['acc'].median():.2f}")


In [61]:
print_stats(results[(results['type'] == 'ft') & (results['repo'] == 'all')])

10, 84.10, 84.10, 84.50


In [62]:
print_stats(results[(results['type'] == 'ft') & (results['repo'] != 'all')])

20, 44.86, 42.85, 45.00


In [63]:
print_stats(results[(results['type'] == '_conf') & (results['repo'] != 'all')])

10, 53.54, 55.20, 50.00


In [64]:
print_stats(results[(results['type'] == '_conf') & (results['repo'] == 'all')])

10, 34.60, 34.60, 35.00


In [65]:
print_stats(results[(results['type'] == '_length_label') & (results['repo'] != 'all')])

10, 54.00, 55.20, 51.50


In [66]:
print_stats(results[(results['type'] == '_length_label') & (results['repo'] == 'all')])

10, 34.80, 34.80, 35.00


In [67]:
print_stats(results[(results['type'] == '_length_input') & (results['repo'] != 'all')])

10, 49.71, 49.40, 46.00


In [68]:
print_stats(results[(results['type'] == '_length_input') & (results['repo'] == 'all')])

10, 35.10, 35.10, 35.00


In [69]:
print_stats(results[(results['type'] == 'bt') & (results['repo'] != 'all')])

10, 53.75, 54.10, 47.50


In [70]:
print_stats(results[(results['type'] == 'bt') & (results['repo'] == 'all')])

0, nan, nan, nan


/tmp/ipykernel_49347/2078206976.py:2: RuntimeWarning: invalid value encountered in double_scalars
  print(f"{len(df)}, {100 * (df['acc'] * df['samples']).sum() / df['samples'].sum():.2f}, {100 * df['acc'].mean():.2f}, {100 * df['acc'].median():.2f}")


In [71]:
print_stats(results[(results['type'] == 'bt') & (results['repo'] != 'all')])

10, 53.75, 54.10, 47.50


In [141]:
results[(results['type'] == 'adapted') & (results['repo'] != 'all') & results['split'].isna()]

,name,repo,acc,#w,samples,time,model,model_address,split,type
22,withAdapter,/data/all/data/qooxdoo/qooxdoo,0.63,18,115,31-01-2022_17-30-12,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
23,withAdapter,/data/all/data/elastic/kibana,0.59,17,51,31-01-2022_17-39-08,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
24,withAdapter,/data/all/data/emberjs/ember.js,0.57,11,47,31-01-2022_17-47-51,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
25,withAdapter,/data/all/data/Encapsule-Annex/onm,1.00,1,39,31-01-2022_18-05-38,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
26,withAdapter,/data/all/data/dcos/dcos-ui,0.92,9,37,31-01-2022_18-22-39,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
27,withAdapter,/data/all/data/svgdotjs/svg.js,0.84,5,31,31-01-2022_18-38-53,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
28,withAdapter,/data/all/data/foam-framework/foam,0.39,13,33,31-01-2022_18-47-10,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
29,withAdapter,/data/all/data/sequelize/sequelize,0.50,12,40,31-01-2022_20-23-31,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
30,normal,/data/all/data/foam-framework/foam,0.42,13,33,31-01-2022_131-01-2022_20-32-47,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
31,withAdapter,/data/all/data/zloirock/core-js,0.59,8,41,01-02-2022_15-31-56,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted


In [135]:
results[(results['type'] == 'adapted') & (results['repo'] != 'all') & results['split'].isna()]

,name,repo,acc,#w,samples,time,model,model_address,split,type
22,withAdapter,/data/all/data/qooxdoo/qooxdoo,0.63,18,115,31-01-2022_17-30-12,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
23,withAdapter,/data/all/data/elastic/kibana,0.59,17,51,31-01-2022_17-39-08,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
24,withAdapter,/data/all/data/emberjs/ember.js,0.57,11,47,31-01-2022_17-47-51,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
25,withAdapter,/data/all/data/Encapsule-Annex/onm,1.00,1,39,31-01-2022_18-05-38,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
26,withAdapter,/data/all/data/dcos/dcos-ui,0.92,9,37,31-01-2022_18-22-39,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
27,withAdapter,/data/all/data/svgdotjs/svg.js,0.84,5,31,31-01-2022_18-38-53,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
28,withAdapter,/data/all/data/foam-framework/foam,0.39,13,33,31-01-2022_18-47-10,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
29,withAdapter,/data/all/data/sequelize/sequelize,0.50,12,40,31-01-2022_20-23-31,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
30,normal,/data/all/data/foam-framework/foam,0.42,13,33,31-01-2022_131-01-2022_20-32-47,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
31,withAdapter,/data/all/data/zloirock/core-js,0.59,8,41,01-02-2022_15-31-56,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted


In [35]:
from matplotlib import pyplot as plt

In [36]:
plt.scatter(inc_2['samples'], inc_2['acc'], label='included')
plt.scatter(exc_2['samples'], exc_2['acc'], label='excluded')
plt.scatter(good_2['samples'], good_2['acc'], label='tuned')
plt.scatter(adapter['samples'], adapter['acc'], label='tuned with adapter')
plt.legend()

NameError: name 'inc_2' is not defined

In [ ]:
diff_inc = inc_2.set_index('repo').join(inc_2.set_index('repo')['acc'] - exc_2.set_index('repo')['acc'], rsuffix='ـdiff')

In [ ]:
diff_good = good_2.set_index('repo').join(good_2.set_index('repo')['acc'] - exc_2.set_index('repo')['acc'], rsuffix='ـdiff')

In [ ]:
diff_adapter = adapter.set_index('repo').join(adapter.set_index('repo')['acc'] - exc_2.set_index('repo')['acc'], rsuffix='ـdiff')

In [ ]:
plt.scatter(diff_inc['samples'], diff_inc['acc'], label='included')
plt.scatter(diff_adapter['samples'], diff_adapter['acc'], label='adapter')
plt.legend()

In [ ]:
plt.scatter(diff_good['samples'], diff_good['acc'], label='good')


In [ ]:
plt.scatter(diff_adapter['samples'], diff_adapter['acc'], label='adapter')
